<a href="https://colab.research.google.com/github/solomontessema/building-ai-agents/blob/main/notebooks/1.5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table>
  <tr>
    <td><img src="https://ionnova.com/img/ionnova_logo_name_2.png" width="120px"></td>
    <td><h1>LangChain Agent with Multiple Tools</h1></td>
  </tr>
</table>

In [ ]:
!pip install -qU langchain==1.1.0 langchain-openai==1.1.0 python-dotenv==1.1.1

In [ ]:
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from langchain_core.tools import Tool
from datetime import datetime
import pytz # Provides the required timezone data.
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)


def get_current_time(city: str) -> str:
    utc_now = datetime.now(pytz.utc)
    city_key = city.lower().strip()

    if "new york" in city_key:
        tz_name = 'America/New_York'
    elif "london" in city_key:
        tz_name = 'Europe/London'
    elif "tokyo" in city_key:
        tz_name = 'Asia/Tokyo'
    else:
        return f"Time service unavailable for {city}."

    target_tz = pytz.timezone(tz_name)
    local_time = utc_now.astimezone(target_tz)
    return str(local_time)

time_tool = Tool(
    name="get_current_time",
    func=get_current_time,
    description="Useful to answers current time in some cities."
)


def simple_math(expression: str) -> str:
    try:
        result = eval(expression)
        return str(result)
    except Exception:
        return "Invalid mathematical expression."

math_tool = Tool(
    name="simple_math",
    func=simple_math,
    description="Useful for calculating simple math."
)

agent = create_agent(
    model=llm,
    tools= [time_tool, math_tool],
    system_prompt="You are a helpful assistant. You must use the provided tools"
)


## Use the Agent

In [ ]:
response=agent.invoke(
    {"messages": [{"role": "user", "content": "what 9 * 19"}]}
)
agent_answer = response["messages"][-1]
print(agent_answer.content)

In [ ]:
# Use the agent
response=agent.invoke(
    {"messages": [{"role": "user", "content": "what time is now in Tokyo?"}]}
)
agent_answer = response["messages"][-1]
print(agent_answer.content)